# Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

# Links

[[citizens_st_mobile]]<br>
[[subscriptions_st_mobile]]<br>
[[citizens_dir_mobile]]<br>
[[subscriptions_st_mobile]]<br>
[[citizens_dir_mobile]]



# flats_with_activated

In [30]:
query_text = """--sql
    CREATE TABLE db1.t_flats_with_activated 
    (
        `report_date` Date,
        `flat_uuid` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)

""


In [35]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_flats_with_activated_mv 
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 20 MINUTE TO db1.t_flats_with_activated AS
    SELECT 
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.flat_uuid AS flat_uuid
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
            ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
            AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
    LEFT JOIN db1.citizens_dir_mobile_ch AS citizens_dir_mobile 
            ON citizens_st_mobile.`citizen_id` = citizens_dir_mobile.`citizen_id`                            
    WHERE subscriptions_st_mobile.state = 'activated'
    """
ch.query_run(query_text)

""


In [56]:
query_text = """--sql
    SELECT
    *
    FROM db1.t_flats_with_activated
    limit 10
    """
ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 373
 Code: 373. DB::Exception: Session 731644e8-25e2-4877-8da7-837a4347e51d is locked by a concurrent client. (SESSION_IS_LOCKED) (version 25.3.3.42 (official build))


# t_citizens_count_in_flats

In [46]:
query_text = """--sql
    CREATE TABLE db1.t_citizens_count_in_flats 
    (
        `report_date` Date,
        `flat_uuid` String,
        `citizen_id` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)

""


In [49]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_citizens_count_in_flats_mv 
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 23 MINUTE TO db1.t_citizens_count_in_flats AS
    SELECT 
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        citizens_st_mobile.citizen_id AS citizen_id
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
            ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
            AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
    LEFT JOIN db1.citizens_dir_mobile_ch AS citizens_dir_mobile 
            ON citizens_st_mobile.`citizen_id` = citizens_dir_mobile.`citizen_id`                            
    WHERE subscriptions_st_mobile.state = 'activated'
    """
ch.query_run(query_text)

""


# t_subscribtions_citizens_by_companies_and_cities

In [ ]:
query_text = """--sql
SELECT 
    citizens_st_mobile.report_date AS report_date,
    citizens_st_mobile.flat_uuid AS flat_uuid,
    citizens_st_mobile.citizen_id AS citizen_id
FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
        ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
        AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
LEFT JOIN db1.citizens_dir_mobile_ch AS citizens_dir_mobile 
        ON citizens_st_mobile.`citizen_id` = citizens_dir_mobile.`citizen_id`                            
WHERE subscriptions_st_mobile.state = 'activated'
LIMIT 10
"""
ch.query_run(query_text)

___

In [ ]:
query_text = """--sql
SELECT
    
FROM (SELECT 
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        citizens_st_mobile.citizen_id AS citizen_id
        if(subscriptions_st_mobile.state = 'activated',1,0) AS if_activated
        max(if_activated) OVER (partition by flat_uuid, report_date)
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.subscriptions_st_mobile_ch AS subscriptions_st_mobile 
            ON citizens_st_mobile.`citizen_id` = subscriptions_st_mobile.`citizen_id`
            AND citizens_st_mobile.`report_date` = subscriptions_st_mobile.`report_date`
    LEFT JOIN db1.citizens_dir_mobile_ch AS citizens_dir_mobile 
            ON citizens_st_mobile.`citizen_id` = citizens_dir_mobile.`citizen_id`)                            
LIMIT 10
"""

___

In [54]:
query_text = """--sql
SELECT
    flats_with_activated.report_date,
    COUNT(citizens_count.citizen_id) as citizen_id_in_flat_with_subscriptions
FROM db1.t_flats_with_activated AS flats_with_activated
LEFT JOIN db1.t_citizens_count_in_flats AS citizens_count
    ON flats_with_activated.report_date = citizens_count.report_date 
    AND flats_with_activated.flat_uuid = citizens_count.flat_uuid
GROUP BY flats_with_activated.report_date
WHERE flats_with_activated.report_date = dateTrunc('day', report_date) 
    AND flats_with_activated >= '2025-01-01'
LIMIT 10
"""
ch.query_run(query_text)

KeyboardInterrupt: 

In [48]:
query_text = """
    DROP TABLE db1.t_citizens_count_in_flats_mv
    """
ch.query_run(query_text)

,elapsed_ns,query_id
0,1287232,e0d6b149-4323-4b7c-bf9f-c83e752e2dca
